In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mosaic24-ps1/sample_submission.csv
/kaggle/input/mosaic24-ps1/train.csv
/kaggle/input/mosaic24-ps1/test.csv


In [3]:
import math
import datetime as dt

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

2024-03-10 15:37:56.807053: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 15:37:56.807158: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 15:37:56.928852: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train = pd.read_csv("/kaggle/input/mosaic24-ps1/train.csv")
test = pd.read_csv("/kaggle/input/mosaic24-ps1/test.csv")

In [5]:
# Pivot the DataFrame
df_closing = train.pivot(index='Date', columns='Company', values=['Close'])

# Flatten the MultiIndex columns
df_closing.columns = ['_'.join(col).strip() for col in df_closing.columns.values]

# Reset index for better structure
df_closing.reset_index(inplace=True)

# Display the result
df_closing.head()

,Date,Close_Company_0,Close_Company_1,Close_Company_2,Close_Company_3,Close_Company_4,Close_Company_5
0,2033-03-01,161.915062,113.066659,50.094499,98.811966,8873.173069,135.882546
1,2033-03-05,163.108507,110.561246,49.250071,96.594597,8794.758109,135.970813
2,2033-03-06,161.716298,109.651386,48.878616,93.602065,8616.178598,136.070561
3,2033-03-07,148.051907,109.723772,48.670564,90.718080,8679.165970,135.772818
4,2033-03-08,148.483368,109.118052,47.642654,90.771456,8729.613528,137.854360


In [6]:
df_closing_company1 = df_closing[['Date', 'Close_Company_1']]

In [7]:
print("DataFrame for Company 1:")
print(df_closing_company1.head())
print()

DataFrame for Company 1:
         Date  Close_Company_1
0  2033-03-01       113.066659
1  2033-03-05       110.561246
2  2033-03-06       109.651386
3  2033-03-07       109.723772
4  2033-03-08       109.118052



In [8]:
df_closing_company1.loc[:, 'Date'] = pd.to_datetime(df_closing_company1['Date'], utc=True)


In [9]:
print("Starting date: ", df_closing_company1.iloc[0, 0])
print("Ending date: ", df_closing_company1.iloc[-1, 0])
print("Duration: ", df_closing_company1.iloc[-1, 0] - df_closing_company1.iloc[0, 0])

Starting date:  2033-03-01 00:00:00+00:00
Ending date:  2038-02-28 00:00:00+00:00
Duration:  1825 days 00:00:00


In [10]:
from datetime import datetime
import pytz

# Convert the string to a timezone-aware Timestamp object
date_threshold_str = '2033-06-01'
date_threshold = pd.Timestamp(datetime.strptime(date_threshold_str, '%Y-%m-%d'), tz=pytz.UTC)

# Filter the DataFrame based on the condition
stock_df_close_last_1_year = df_closing_company1[df_closing_company1['Date'] > date_threshold]

# Make a copy of the filtered DataFrame
stock_df_close_last_1_year_original_gt = stock_df_close_last_1_year.copy()

# Print the total data for prediction
print("Total data for prediction: ", stock_df_close_last_1_year.shape[0])

Total data for prediction:  1173


In [11]:
fig = px.line(stock_df_close_last_1_year, x=stock_df_close_last_1_year.Date, y=stock_df_close_last_1_year.Close_Company_1,labels={'Date':'Date','Close_Company_1':'Close_Company_1'})

fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')

fig.update_layout(title_text='Considered period to predict Stock close price', plot_bgcolor='white', font_size=15, font_color='black')

fig.update_xaxes(showgrid=False)

fig.update_yaxes(showgrid=False)

fig.show()

In [12]:
training_size=int(len(stock_df_close_last_1_year)*0.80)

test_size=len(stock_df_close_last_1_year)-training_size

train_data,test_data = stock_df_close_last_1_year[0:training_size], stock_df_close_last_1_year[training_size:len(stock_df_close_last_1_year)]

print("train_data: ", train_data.shape)  # (201, 2)

print("test_data: ", test_data.shape) # (51, 2)

train_data:  (938, 2)
test_data:  (235, 2)


In [13]:
del train_data['Date']

del test_data['Date']

In [14]:
scaler=MinMaxScaler(feature_range=(0,1))

train_data=scaler.fit_transform(np.array(train_data).reshape(-1,1))

test_data = scaler.transform(np.array(test_data).reshape(-1,1))

print(train_data.shape)

(938, 1)


In [15]:
def create_sliding_window(dataset, time_step=200):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [16]:
time_step = 200

X_train, y_train = create_sliding_window(train_data, time_step)

X_test, y_test = create_sliding_window(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (737, 200)
y_train:  (737,)
X_test:  (34, 200)
y_test (34,)


In [17]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (737, 200, 1)
X_test:  (34, 200, 1)


In [23]:
model=Sequential()

model.add(GRU(32,return_sequences=True,input_shape=(time_step,1)))

model.add(GRU(32,return_sequences=True))

model.add(GRU(32))

model.add(Dropout(0.20))

model.add(Dense(1))

model.compile(loss='mean_squared_error',optimizer='adam')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [24]:
history = model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=200, batch_size=32, verbose=1)

Epoch 1/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.1891 - val_loss: 0.0182
Epoch 2/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0184 - val_loss: 0.0057
Epoch 3/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0090 - val_loss: 0.0030
Epoch 4/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0070 - val_loss: 0.0045
Epoch 5/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0052 - val_loss: 0.0077
Epoch 6/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0051 - val_loss: 0.0062
Epoch 7/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0043 - val_loss: 0.0048
Epoch 8/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0050 - val_loss: 0.0049
Epoch 9/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0046 - val_loss: 0.0034
Epoch 10/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0049 - val_loss: 0.0046
Epoch 11/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0044 - val_loss: 0.0034
Epoch 12/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

In [28]:
train_predict=model.predict(X_train)

test_predict=model.predict(X_test)

train_predict.shape, test_predict.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


((737, 1), (34, 1))

In [29]:
train_predict = scaler.inverse_transform(train_predict)

test_predict = scaler.inverse_transform(test_predict)

original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 

original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [30]:
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain, train_predict)))

print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))

print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))

print("-------------------------------------------------------------------------------------")

print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))

print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))

print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  2.2701968247002395
Train data MSE:  5.153793622879051
Train data MAE:  1.7070777564179793
-------------------------------------------------------------------------------------
Test data RMSE:  1.9353691994373499
Test data MSE:  3.7456539381307685
Test data MAE:  1.4711315439127228


In [31]:
# Reshape test data to use as input for prediction
x_input = test_data[len(test_data) - time_step:]
x_input = (np.array(x_input)).reshape(1,time_step,-1)
print("x_input.shape ", x_input.shape)
# print("x_input", x_input)
print('###########################################################################')

from numpy import array

# Initialize variables
temp_input = list(x_input)
lst_output = []
n_steps = time_step
pred_days = 96  # Predict for 96 days

for _ in range(pred_days):
    if len(temp_input[0]) > time_step:
        x_input = array(temp_input[0][1:])  # Remove the first element from temp_input
        x_input = x_input.reshape((1, n_steps, -1))
        y_pred = model.predict(x_input, verbose=0)
        
        temp_input[0] = np.append(temp_input[0][1:], y_pred)  # Append the predicted value to temp_input
        lst_output.append(y_pred[0].tolist())
    else:
        # Reshape input for prediction if temp_input length is less than time_step
        x_input = array(temp_input[0])
        x_input = x_input.reshape((1, n_steps, -1))
        y_pred = model.predict(x_input, verbose=0)
        
        temp_input[0] = np.append(temp_input[0], y_pred)  # Append the predicted value to temp_input
        lst_output.append(y_pred[0].tolist())

print("Output of predicted next days: ", len(lst_output))

x_input.shape  (1, 200, 1)
###########################################################################
Output of predicted next days:  96


In [32]:
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  96


In [33]:
# lst_output

In [34]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
# print(last_days)
# print(day_pred)

In [35]:
temp_matrix = np.empty((len(last_days)+pred_days, 1))

temp_matrix[:] = np.nan

# temp_matrix

In [36]:
# last_original_days_value.shape

In [37]:
stock_df_close_last_1_year_original_gt.iloc[-time_step:]['Close_Company_1'].shape

(200,)

In [39]:
lst_output = np.array(lst_output)
lst_output.shape

(96, 1)

In [40]:
lst_output

array([[0.66520333],
       [0.67716658],
       [0.68728888],
       [0.6977514 ],
       [0.70817113],
       [0.71848768],
       [0.72870553],
       [0.73880506],
       [0.74877417],
       [0.75860357],
       [0.76828551],
       [0.77781296],
       [0.78717911],
       [0.79637742],
       [0.8054018 ],
       [0.81424671],
       [0.82290697],
       [0.83137798],
       [0.839656  ],
       [0.84773767],
       [0.85562038],
       [0.86330205],
       [0.87078142],
       [0.87805778],
       [0.88513088],
       [0.89200127],
       [0.89866996],
       [0.90513837],
       [0.91140842],
       [0.91748261],
       [0.92336357],
       [0.92905438],
       [0.93455851],
       [0.93987966],
       [0.94502163],
       [0.94998872],
       [0.95478511],
       [0.95941508],
       [0.96388328],
       [0.96819413],
       [0.97235227],
       [0.97636235],
       [0.98022902],
       [0.98395681],
       [0.98755038],
       [0.99101424],
       [0.9943527 ],
       [0.997

In [41]:
last_original_days_value = temp_matrix

next_predicted_days_value = temp_matrix

last_original_days_value[0:time_step] = (np.array((stock_df_close_last_1_year_original_gt.iloc[-time_step:]['Close_Company_1']))).reshape(-1,1).tolist()

# Assign values to next_predicted_days_value
next_predicted_days_value[time_step:] = (scaler.inverse_transform(lst_output)).tolist()[0]

In [42]:
# last_original_days_value = temp_matrix.copy()
# next_predicted_days_value = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]


In [43]:
# Assuming scaler is your MinMaxScaler object
# Inverse transform lst_output
new_predicted_values = scaler.inverse_transform(np.array(lst_output).reshape(-1, 1))

# Convert predicted_values to a 1D array
new_predicted_values = new_predicted_values.flatten()


In [44]:
new_predicted_values

array([102.91416361, 103.93115313, 104.79164453, 105.68105814,
       106.56683366, 107.44383829, 108.31245202, 109.17100784,
       110.01847715, 110.85406951, 111.67712621, 112.48704935,
       113.28326129, 114.06520468, 114.83236268, 115.58426406,
       116.320468  , 117.04058434, 117.74429386, 118.43131282,
       119.10141827, 119.75443286, 120.39025019, 121.00880946,
       121.61009039, 122.19413859, 122.76104019, 123.31091682,
       123.8439306 , 124.36029435, 124.86023101, 125.34400408,
       125.81190742, 126.2642552 , 126.7013717 , 127.12362173,
       127.53136   , 127.92495131, 128.30479089, 128.6712537 ,
       129.02473496, 129.36562988, 129.69433371, 130.01123151,
       130.31671853, 130.61117983, 130.89498026, 131.16850489,
       131.43211854, 131.68616578, 131.93100128, 132.16696961,
       132.39439503, 132.61359172, 132.82487381, 133.02855547,
       133.22492043, 133.41424232, 133.59679475, 133.77284121,
       133.94263503, 134.10641945, 134.26442752, 134.41

In [45]:
print(len(next_predicted_days_value))

296


In [46]:
# next_predicted_days_value

In [47]:
# Convert next_predicted_days_value to a DataFrame
predictions_df = pd.DataFrame(next_predicted_days_value, columns=['Predictions1'])

# Save the DataFrame to a CSV file
predictions_df.to_csv('predictions1.csv', index=False)

In [48]:
# final prediction
# scaler is your MinMaxScaler object
# Inverse transform lst_output
new_predicted_values = scaler.inverse_transform(np.array(lst_output).reshape(-1, 1))

# Convert predicted_values to a 1D array
new_predicted_values = new_predicted_values.flatten()


# Convert next_predicted_days_value to a DataFrame
new_predictions_df = pd.DataFrame(new_predicted_values, columns=['Predictions1'])

# Save the DataFrame to a CSV file
new_predictions_df.to_csv('newpredictions1.csv', index=False)